# How to obtain the OTU tables for freshwater analyses

User request:
```
I want to download OTU table in biom format of each analysis in freshwater, I am wondering how to obtain them by API.
```

In [20]:
# Requirements
!pip install requests

## Obtain the analysis for freshwater

In [21]:
import requests
import csv

API = "https://www.ebi.ac.uk/metagenomics/api/v1"

def fetch(analyses_accessions, url=""):
    if not url:
        # First request!
        # Note the filters
        response = requests.get(API + "/analyses", params={
            "lineage": "root:Environmental:Aquatic:Freshwater",
            "experiment_type": "amplicon"
        })
    else:
        response = requests.get(url)

    response_data = response.json()
    next_url = response_data.get('links', {}).get('next')

    for entry in response_data.get('data'):
        analyses_accessions.append(entry.get('id'))
    
    # keep getting the accessions
    if next_url and "page=5" not in next_url: # artifical stop in page 5
        print(f"fetching: {next_url}")
        fetch(analyses_accessions, next_url)

# start the fetch process
analyses_accessions = []
fetch(analyses_accessions)

fetching: https://www.ebi.ac.uk/metagenomics/api/v1/analyses?experiment_type=amplicon&lineage=root%3AEnvironmental%3AAquatic%3AFreshwater&page=2
fetching: https://www.ebi.ac.uk/metagenomics/api/v1/analyses?experiment_type=amplicon&lineage=root%3AEnvironmental%3AAquatic%3AFreshwater&page=3
fetching: https://www.ebi.ac.uk/metagenomics/api/v1/analyses?experiment_type=amplicon&lineage=root%3AEnvironmental%3AAquatic%3AFreshwater&page=4


## Get the download links for the .biom files

In [22]:
# Now we need to go one more level to the download files for the analysis
for accession in analyses_accessions:
    print(f"Getting download files for {accession}")
    response = requests.get(API + "/analyses/" + accession + "/downloads")
    for download_entry in response.json().get("data"):
        attrs = download_entry.get("attributes")
        if attrs.get("file-format").get("extension") == "biom":
            # URL of the file
            print(download_entry.get("links").get("self"))
    break # artifical interruption

Getting download files for MGYA00429528
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00429528/file/ERR1553864_FASTQ_SSU_OTU_TABLE_HDF5.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00429528/file/ERR1553864_FASTQ_SSU_OTU_TABLE_JSON.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00429528/file/ERR1553864_FASTQ_ITS_ITSoneDB_OTU_TABLE_HDF5.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00429528/file/ERR1553864_FASTQ_ITS_ITSoneDB_OTU_TABLE_JSON.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00429528/file/ERR1553864_FASTQ_ITS_UNITE_OTU_TABLE_HDF5.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00429528/file/ERR1553864_FASTQ_ITS_UNITE_OTU_TABLE_JSON.biom
